# Neural Networks retrieval on the Satellite Observations Dataset

In this final step we develop a model to estimate soil moisture using neural networks

Libraries we need are imported first

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
import xarray as xr
from scipy.interpolate import griddata
from scipy import interpolate
import statsmodels.api as sm

import torch
from torch import nn, optim
from torch.optim import SGD


# from torch.optim import RMSprop
# from torch.utils.data import TensorDataset
# from torchmetrics import (MeanAbsoluteError, R2Score)
# # from torchinfo import summary
# from torchvision.io import read_image
# from pytorch_lightning import Trainer
# from pytorch_lightning.loggers import CSVLogger

Data used in this exercise is stored in the DATA folder

In [6]:
thefile = '../../../DATA/TPdata_199301'
data = pd.read_csv(thefile, sep='\t', header=None)
data.columns = ["cellNr", "latitude", "longitude", "backscatter","emissivity_v","emissivity_h","ts_amplitude","ndvi","lmd_soilWetness"]
data = data.replace('   NaN',pd.NA)
data['backscatter'] = pd.to_numeric(data['backscatter'])
data['emissivity_v'] = pd.to_numeric(data['emissivity_v'])
data['emissivity_h'] = pd.to_numeric(data['emissivity_h'])
data['ts_amplitude'] = pd.to_numeric(data['ts_amplitude'])
data['ndvi'] = pd.to_numeric(data['ndvi'])
data['lmd_soilWetness'] = pd.to_numeric(data['lmd_soilWetness'])
data = data.dropna()

In [122]:
thres = 20000

In [124]:
back = data['backscatter'].values[0:thres]
lats = data['latitude'].values[0:thres]
lons = data['longitude'].values[0:thres]
emissivity_v = data['emissivity_v'].values[0:thres]
emissivity_h = data['emissivity_h'].values[0:thres]
ts_amplitude = data['ts_amplitude'].values[0:thres]
ndvi = data['ndvi'].values[0:thres]
lmd_soilWetness = data['lmd_soilWetness'].values[0:thres]
Yi = np.linspace(np.min(lats.data),float(np.max(lats.data)),180)
Xi = np.linspace(np.min(lons.data),float(np.max(lons.data)),360)
X, Y = np.meshgrid(Xi,Yi)
points = list(zip(lons, lats))
back_gridded = griddata(points, back, (X, Y), method='nearest')
emissivity_v_gridded = griddata(points, emissivity_v, (X, Y), method='nearest')
emissivity_h_gridded = griddata(points, emissivity_h, (X, Y), method='nearest')
ts_amplitude_gridded = griddata(points, ts_amplitude, (X, Y), method='nearest')
ndvi_gridded = griddata(points, ndvi, (X, Y), method='nearest')
lmd_soilWetness_gridded = griddata(points, lmd_soilWetness, (X, Y), method='nearest')

## Neural Network

In [127]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(5,100)  
        self.fc2 = nn.Linear(100,1)  

    def forward(self, x):
        x = torch.relu(self.fc1(x))  
        x = self.fc2(x)               
        return x

In [141]:
alles = np.stack((back,emissivity_v,emissivity_h,ts_amplitude,ndvi),axis=1)

In [135]:
X_train = torch.tensor(alles).to(torch.float32)
y_train = torch.tensor(lmd_soilWetness).to(torch.float32)

In [137]:
model = SimpleNN()  
criterion = nn.MSELoss()  
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [139]:
for epoch in range(100):  
    model.train() 

    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train)  
    
    # Backward pass and optimize
    optimizer.zero_grad()  
    loss.backward()  
    optimizer.step()  

    if (epoch + 1) % 10 == 0:  
        print(f'Epoch [{epoch + 1}/100], Loss: {loss.item():.4f}')

/opt/anaconda3/lib/python3.12/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([20000])) that is different to the input size (torch.Size([20000, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [10/100], Loss: 26426.5234
Epoch [20/100], Loss: 304.7427
Epoch [30/100], Loss: 3.5792
Epoch [40/100], Loss: 0.1070
Epoch [50/100], Loss: 0.0670
Epoch [60/100], Loss: 0.0665
Epoch [70/100], Loss: 0.0665
Epoch [80/100], Loss: 0.0665
Epoch [90/100], Loss: 0.0665
Epoch [100/100], Loss: 0.0665


## Performance of the Neural Network